In [ ]:
### Uncomment the following lines to install the required packages###
# For Microsoft Entra ID token authentication support 
# ! pip install azure-identity 
# # For Azure AI Projects client library from Azure AI Foundry SDK packages and services
# ! pip install azure-ai-projects
# # For Semantic Kernel client library from Azure AI Foundry SDK packages and services     
# ! pip install semantic-kernel

In [ ]:
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool
import os
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion  
from semantic_kernel.planners import SequentialPlanner
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from typing import Annotated

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str='<Your Project connection string from Azure AI Foundry>',
)

In [ ]:
bing_connection = project_client.connections.get(
    connection_name='<Your Bing connection name>',
)
conn_id = bing_connection.id

In [ ]:
class Agents:
    @kernel_function(
        description="This function will be used to use an azure ai agent with web grounding capability using Bing Search API",
        name="WebSearchAgent"
    )
    def web_search_agent(
        self,
        query: Annotated[str, "The user query for which the contextual information needs to be fetched from the web"]
        
    ) -> Annotated[str, "The response from the web search agent"]:
        conn_id = bing_connection.id
        bing = BingGroundingTool(connection_id=conn_id)
        
        agent = project_client.agents.create_agent(
        model="gpt-4o",
            name="bing-assistant",
            instructions="You are a helpful assistant",
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"},
        )
        
        thread = project_client.agents.create_thread()
            
        message = project_client.agents.create_message(
                thread_id=thread.id,
                role="user",
                content=query,
            )
            
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
            
        messages = project_client.agents.list_messages(thread_id=thread.id)
        
        print("Response from the web search agent:")
        print("\n")
        print(messages.data[0].content[0].text.value)
            
        return messages.data[0].content[0].text.value
       
    
    @kernel_function(
       description="This function will use an azure ai agent to prepare a script for a news reporter based on latest information for a specific topic",
         name="NewsReporterAgent"
   )
    def news_reporter_agent(
        self,
        topic: Annotated[str, "The topic for which the latest information/news has been fetched"],
        latest_news: Annotated[str,"The latest information for a specific topic"]
    ) -> Annotated[str, "the response from the NewsReporterAgent which is the script for a news reporter"]:

        agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="news-reporter",
        instructions="""You are a helpful assistant that is meant to prepare a script for a news reporter based on the latest information for a specific topic both of which you will be given.
            The news channel is named MSinghTV and the news reporter is named John. You will be given the topic and the latest information for that topic. Prepare a script for the news reporter John based on the latest information for the topic.""",
            headers={"x-ms-enable-preview": "true"},
        )
        
        thread = project_client.agents.create_thread()
            
        message = project_client.agents.create_message(
                thread_id=thread.id,
                role="user",
                content=f"""The topic is {topic} and the latest information is {latest_news}""",
            )
            
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
            
        messages = project_client.agents.list_messages(thread_id=thread.id)
        
        print("Script for the news reporter:")
        print("\n")    
        print(messages.data[0].content[0].text.value)
            
        return messages.data[0].content[0].text.value

In [ ]:
kernel = Kernel()

service_id = "default"

kernel.add_service(
    AzureChatCompletion(service_id=service_id,
                        api_key="9v2KveJczflDWtVeDLFvJj5PNIZq3LzfNaxwHHYbcyzjYPiSLzgFJQQJ99BDACfhMk5XJ3w3AAAAACOGk0ho",
                        deployment_name="gpt-4o",
                        endpoint = "https://ai-pranabp6630ai269359085220.openai.azure.com/"
    )
)

In [ ]:
planner = SequentialPlanner(
    kernel,
    service_id
)

In [ ]:
agents_plugin = kernel.add_plugin(Agents(), "Agents")

In [ ]:
goal = f"prepare a news script for John on latest news for the world?"

In [ ]:
async def call_planner():
    return await planner.create_plan(goal)

sequential_plan = await call_planner()

print("The plan's steps are:")
for step in sequential_plan._steps:
    print(
        f"- {step.description.replace('.', '') if step.description else 'No description'} using {step.metadata.fully_qualified_name} with parameters: {step.parameters}"
    )

In [ ]:
async def generate_answer():
    return await sequential_plan.invoke(kernel)

result = await generate_answer()

print(result)